In [ ]:
# Environment and Agent Definitions
import numpy as np
import random

# Define agent class to hold state
class QTableAgent:
    def __init__(self, agent_id):
        self.agent_id = agent_id
        self.has_item = 0        # 1 if carrying an item, else 0
        self.direction = None    # True if heading A->B, False if heading B->A
        self.position = None
        self.previous_position = None
        self.local_mask = 0      # 8-bit mask for opposite-direction neighbors
        self.update_order = None # order index for central clock updates
        self.collision_penalty = 0
        self.completed_round_trip = False

# Grid world environment for multi-agent shuttle
class GridWorldEnvironment:
    def __init__(self, n=5, m=5, num_agents=4, food_source_location=None, nest_location=None):
        self.n = n
        self.m = m
        self.num_agents = num_agents
        # Set A (food source) and B (nest) locations
        self.food_source_location = food_source_location if food_source_location is not None else (0, 0)
        self.nest_location = nest_location if nest_location is not None else (n-1, m-1)
        # Initialize agents list
        self.agents = [QTableAgent(i) for i in range(num_agents)]
        # Define relative positions for the 8 neighboring directions (N, NE, E, SE, S, SW, W, NW)
        self.directions = [(-1, 0), (-1, 1), (0, 1), (1, 1),
                            (1, 0), (1, -1), (0, -1), (-1, -1)]
        # Central clock for coordinated updates
        self.clock = 0
        self.update_sequence = list(range(num_agents))  # default round-robin order
        # Collision and round-trip tracking
        self.collision_count = 0
        self.collision_penalty_value = -50  # penalty applied to each agent in a collision
        self.round_trip_count = 0
        self.agent_states = [0] * num_agents       # progress state (0->3) for each agent's round trip
        self.agent_round_trips = [False] * num_agents  # whether each agent completed a round trip
        # Place agents at start positions
        self._reset()  # randomize initial agent placements at A or B

    def _reset(self):
        """Reset agents to random start at A or B, and reset environment counters."""
        # Randomize each agent's starting position: 50% at A (with item), 50% at B (empty)
        for agent in self.agents:
            if random.random() < 0.5:
                agent.position = self.food_source_location
                agent.has_item = 1
                agent.direction = True   # carrying item, so heading toward B
            else:
                agent.position = self.nest_location
                agent.has_item = 0
                agent.direction = False  # not carrying, so heading toward A
            agent.previous_position = None
            agent.local_mask = 0
            agent.collision_penalty = 0
            agent.completed_round_trip = False
        # Reset global counters
        self.clock = 0
        self.update_sequence = list(range(self.num_agents))  # round-robin order
        self.collision_count = 0
        self.round_trip_count = 0
        self.agent_states = [0] * self.num_agents
        self.agent_round_trips = [False] * self.num_agents

    def get_next_agent(self):
        """Return the next agent ID to act, based on the central clock (round-robin)."""
        agent_id = self.update_sequence[self.clock % self.num_agents]
        self.clock += 1
        return agent_id

    def get_local_mask(self, agent_id):
        """
        Compute an 8-bit mask for agent agent_id indicating which of the 8 neighboring cells
        contain an agent moving in the opposite direction (potential head-on collision threat).
        """
        agent = self.agents[agent_id]
        x, y = agent.position
        mask = 0
        # Check all neighbors
        for i, (dx, dy) in enumerate(self.directions):
            nx, ny = x + dx, y + dy
            if 0 <= nx < self.n and 0 <= ny < self.m:
                for other in self.agents:
                    if other.agent_id != agent_id:
                        if other.position == (nx, ny) and other.direction != agent.direction:
                            # Neighbor cell occupied by an opposite-direction agent
                            mask |= (1 << i)
        return mask

    def get_state(self, agent_id):
        """Return the 15-dimensional state vector for the given agent."""
        agent = self.agents[agent_id]
        # Update the agent's local mask (for visualization/debugging if needed)
        agent.local_mask = self.get_local_mask(agent_id)
        # Construct state: position (x,y), A coords, B coords, carry flag, and 8 neighbor flags
        x, y = agent.position
        state = [
            x, y,
            self.food_source_location[0], self.food_source_location[1],
            self.nest_location[0], self.nest_location[1],
            agent.has_item
        ]
        # Append 8 binary flags indicating opposite-direction agents around
        for i in range(8):
            flag = (agent.local_mask >> i) & 1
            state.append(flag)
        return state

    def check_collisions(self):
        """
        Detect head-on collisions: if any non-endpoint cell is occupied by at least one agent with an item 
        and at least one without an item (i.e., agents heading in opposite directions). 
        Apply collision penalty to those agents.
        """
        positions = {}
        # Group agents by their current position
        for agent in self.agents:
            positions.setdefault(agent.position, []).append(agent)
        # Check each position that has more than one agent
        for pos, agents in positions.items():
            if len(agents) < 2:
                continue
            # Ignore collisions at A or B (agents clustering at endpoints is allowed)
            if pos == self.food_source_location or pos == self.nest_location:
                continue
            # Check if there's at least one carrying and one not carrying agent
            carrying = any(a.has_item for a in agents)
            not_carrying = any(not a.has_item for a in agents)
            if carrying and not_carrying:
                # Head-on collision detected
                self.collision_count += 1
                for agent in agents:
                    agent.collision_penalty += self.collision_penalty_value

    def take_action(self, agent_id, action):
        """
        Execute the given action for agent agent_id. 
        Moves the agent, applies pickup/dropoff, updates round-trip state, and returns the reward.
        """
        agent = self.agents[agent_id]
        # Save previous position (for potential collision checks or info)
        agent.previous_position = agent.position
        x, y = agent.position
        # Determine next position based on action
        if action == 0:   # Up
            nx, ny = max(x - 1, 0), y
        elif action == 1: # Down
            nx, ny = min(x + 1, self.n - 1), y
        elif action == 2: # Left
            nx, ny = x, max(y - 1, 0)
        elif action == 3: # Right
            nx, ny = x, min(y + 1, self.m - 1)
        else:
            nx, ny = x, y  # no-op for safety (should not happen)
        next_pos = (nx, ny)
        # Default step penalty
        reward = -1
        # Check for pickup or drop-off events
        if next_pos == self.food_source_location and agent.has_item == 0:
            # Arriving at A without an item -> pick up item
            agent.has_item = 1
            reward += 40   # net +39 for pickup (reward was -1, now -1+40)
        elif next_pos == self.nest_location and agent.has_item == 1:
            # Arriving at B with an item -> drop off
            agent.has_item = 0
            reward += 60   # net +59 for successful delivery
        # Update agent's position
        agent.position = next_pos
        # Update agent's travel direction based on carry status
        agent.direction = True if agent.has_item else False

        # Update agent's round-trip progress state machine:
        # States: 0 (not started or reset), 1 (halfway), 2 (reached opposite end), 3 (completed round trip).
        # A -> B -> A trip tracking
        if self.agent_states[agent_id] == 0 and agent.position == self.food_source_location and agent.has_item:
            self.agent_states[agent_id] = 1  # Picked up at A, heading to B
        elif self.agent_states[agent_id] == 1 and agent.position == self.nest_location and agent.has_item == 0:
            self.agent_states[agent_id] = 2  # Dropped off at B (halfway done A->B->A)
        elif self.agent_states[agent_id] == 2 and agent.position == self.food_source_location and agent.has_item == 0:
            # Came back to A without item -> completed A->B->A round trip
            self.agent_states[agent_id] = 3
            self.agent_round_trips[agent_id] = True
            self.round_trip_count += 1
            agent.completed_round_trip = True

        # B -> A -> B trip tracking
        if self.agent_states[agent_id] == 0 and agent.position == self.nest_location and agent.has_item == 0:
            self.agent_states[agent_id] = 1  # Started at B, heading to A
        elif self.agent_states[agent_id] == 1 and agent.position == self.food_source_location and agent.has_item:
            self.agent_states[agent_id] = 2  # Picked up at A, heading back to B
        elif self.agent_states[agent_id] == 2 and agent.position == self.nest_location and agent.has_item == 0:
            # Returned to B with item delivered -> completed B->A->B round trip
            self.agent_states[agent_id] = 3
            self.agent_round_trips[agent_id] = True
            self.round_trip_count += 1
            agent.completed_round_trip = True

        return reward


In [ ]:
# DQN Agent Setup and Replay Buffer

import torch
import torch.nn as nn
import torch.optim as optim

# Replay buffer for experience replay
class ReplayBuffer:
    def __init__(self, capacity):
        self.buffer = []
        self.capacity = capacity
    def push(self, state, action, reward, next_state, done):
        if len(self.buffer) >= self.capacity:
            # Remove oldest experience if at capacity
            self.buffer.pop(0)
        self.buffer.append((state, action, reward, next_state, done))
    def sample(self, batch_size):
        batch = random.sample(self.buffer, batch_size)
        states      = np.array([exp[0] for exp in batch], dtype=np.float32)
        actions     = np.array([exp[1] for exp in batch], dtype=np.int64)
        rewards     = np.array([exp[2] for exp in batch], dtype=np.float32)
        next_states = np.array([exp[3] for exp in batch], dtype=np.float32)
        dones       = np.array([exp[4] for exp in batch], dtype=np.float32)
        return states, actions, rewards, next_states, dones
    def __len__(self):
        return len(self.buffer)

# Initialize DQN policy network and target network
state_dim = 15   # state features (from environment state vector)
action_dim = 4   # four possible actions
policy_net = nn.Sequential(
    nn.Linear(state_dim, 128),
    nn.ReLU(),
    nn.Linear(128, 128),
    nn.ReLU(),
    nn.Linear(128, action_dim)
)
target_net = nn.Sequential(
    nn.Linear(state_dim, 128),
    nn.ReLU(),
    nn.Linear(128, 128),
    nn.ReLU(),
    nn.Linear(128, action_dim)
)
# Copy weights from policy to target network
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

# DQN hyperparameters
learning_rate = 1e-3
gamma = 0.95             # discount factor
batch_size = 64
target_update_freq = 250 # how often to sync target net (in updates)

optimizer = optim.Adam(policy_net.parameters(), lr=learning_rate)
loss_fn = nn.MSELoss()

# Epsilon-greedy exploration schedule
class EpsilonScheduler:
    def __init__(self, start=1.0, end=0.1, decay=0.995):
        self.epsilon = start
        self.min_epsilon = end
        self.decay = decay
    def get_epsilon(self):
        # Return current epsilon and decay it for next call
        eps = self.epsilon
        self.epsilon = max(self.min_epsilon, self.epsilon * self.decay)
        return eps


In [ ]:
# Training Loop

# Logging utility to record training metrics
class MetricLogger:
    def __init__(self):
        self.rewards = []
        self.collisions = []
        self.round_trips = []
        self.lengths = []
    def log_episode(self, total_reward, collisions, round_trips, length):
        self.rewards.append(total_reward)
        self.collisions.append(collisions)
        self.round_trips.append(round_trips)
        self.lengths.append(length)
    def get_stats(self):
        return {
            "mean_reward": np.mean(self.rewards) if self.rewards else 0.0,
            "mean_collisions": np.mean(self.collisions) if self.collisions else 0.0,
            "mean_round_trips": np.mean(self.round_trips) if self.round_trips else 0.0,
            "mean_length": np.mean(self.lengths) if self.lengths else 0.0
        }

# Training parameters and budgets
num_episodes = 10000
max_steps_per_episode = 120   # safety cap on steps per episode (will often terminate earlier)
buffer_capacity = 50000
step_budget = 1_500_000       # max training steps (across all agents)
collision_budget = 4000       # max total collisions during training
walltime_budget = 600         # max wall-clock time (seconds) for training

# Initialize environment, replay buffer, epsilon scheduler, and metric logger
env = GridWorldEnvironment()
replay_buffer = ReplayBuffer(buffer_capacity)
eps_scheduler = EpsilonScheduler(start=1.0, end=0.1, decay=0.995)
logger = MetricLogger()

# Set random seeds for reproducibility
random.seed(42); np.random.seed(42); torch.manual_seed(42)

# Training loop
import time
start_time = time.time()
total_steps = 0
total_collisions = 0

print("Starting training...")
for episode in range(1, num_episodes + 1):
    # Randomize A and B locations for this episode (ensuring A != B)
    while True:
        a_row = np.random.randint(env.n)
        a_col = np.random.randint(env.m)
        b_row = np.random.randint(env.n)
        b_col = np.random.randint(env.m)
        if (a_row, a_col) != (b_row, b_col):
            break
    env.food_source_location = (a_row, a_col)
    env.nest_location = (b_row, b_col)
    env._reset()

    episode_reward = 0
    episode_collisions = 0
    episode_round_trips = 0
    eps = eps_scheduler.get_epsilon()  # current epsilon for this episode
    done_episode = False

    # Loop for agent moves within the episode
    for step in range(max_steps_per_episode):
        # Each iteration allows all agents to move once (round-robin order)
        for _ in range(env.num_agents):
            agent_id = env.get_next_agent()
            state = env.get_state(agent_id)
            # ϵ-greedy action selection
            if random.random() < eps:
                action = random.randrange(action_dim)
            else:
                # Use policy net to select best action
                state_tensor = torch.tensor(state, dtype=torch.float32)
                q_values = policy_net(state_tensor)
                action = int(torch.argmax(q_values).item())
            # Execute action
            reward = env.take_action(agent_id, action)
            # Check for collisions caused by this action
            env.check_collisions()
            if env.collision_count > episode_collisions:
                # A new collision occurred in this step
                new_collisions = env.collision_count - episode_collisions
                total_collisions += new_collisions
                episode_collisions = env.collision_count
                # Apply collision penalties to agents involved
                for ag in env.agents:
                    if ag.collision_penalty != 0:
                        # Add penalty to moving agent's reward (already included in env.take_action via reward if same agent)
                        if ag.agent_id != agent_id:
                            # If another agent was collided into, include its penalty in total episode reward (it will not act this step)
                            reward += ag.collision_penalty
                        episode_reward += ag.collision_penalty
                        ag.collision_penalty = 0  # reset after applying
            # Observe next state
            next_state = env.get_state(agent_id)
            done = False  # we don't use 'done' for per-agent steps in training
            # Store experience in replay buffer
            replay_buffer.push(state, action, reward, next_state, done)
            # Increment episodic totals
            episode_reward += reward

            # DQN training step: sample a batch and update the network
            if len(replay_buffer) >= batch_size:
                states, actions, rewards, next_states, dones = replay_buffer.sample(batch_size)
                # Compute current Q estimates and target Q values
                state_batch = torch.tensor(states, dtype=torch.float32)
                next_state_batch = torch.tensor(next_states, dtype=torch.float32)
                action_batch = torch.tensor(actions, dtype=torch.int64).unsqueeze(1)
                reward_batch = torch.tensor(rewards, dtype=torch.float32).unsqueeze(1)
                done_batch = torch.tensor(dones, dtype=torch.float32).unsqueeze(1)
                # Current Q values (policy_net) for taken actions
                q_current = policy_net(state_batch).gather(1, action_batch)
                # Compute next Q values from target_net
                q_next = target_net(next_state_batch).max(dim=1, keepdim=True)[0]
                # Compute target Q: r + γ * max Q_next (0 if done)
                q_target = reward_batch + gamma * q_next * (1 - done_batch)
                # Optimize the policy network
                loss = loss_fn(q_current, q_target)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            total_steps += 1
            # Check if any agent finished a round trip in this step
            if env.round_trip_count > 0:
                done_episode = True
            # Check global stopping criteria
            if total_steps >= step_budget or total_collisions >= collision_budget or (time.time() - start_time) >= walltime_budget:
                done_episode = True
            if done_episode:
                break  # break out of for _ in range(num_agents)
        # Sync target network periodically
        if total_steps % target_update_freq == 0:
            target_net.load_state_dict(policy_net.state_dict())
        if done_episode or step == max_steps_per_episode - 1:
            # Record round trips completed in this episode (if any)
            episode_round_trips = env.round_trip_count
            break  # break out of step loop

    # Log metrics for this episode
    logger.log_episode(episode_reward, episode_collisions, episode_round_trips, step+1)
    # Print progress every 10 episodes
    if episode % 10 == 0:
        print(f"Episode {episode:3d}: Reward={episode_reward:.0f}, Collisions={episode_collisions}, RoundTrips={episode_round_trips}, Epsilon={eps:.2f}")

    # Check training termination conditions after each episode
    if total_steps >= step_budget or total_collisions >= collision_budget or (time.time() - start_time) >= walltime_budget:
        print("Stopping training early due to budget limit reached.")
        break

# Save trained model weights
torch.save(policy_net.state_dict(), "trained_model.pth")
print("Training completed.")
stats = logger.get_stats()
print(f"Average Reward: {stats['mean_reward']:.2f}, Average Collisions: {stats['mean_collisions']:.2f}, Average RoundTrips: {stats['mean_round_trips']:.2f}")


In [ ]:
# Evaluation of Trained Policy

# Load the trained model for evaluation
trained_model = nn.Sequential(
    nn.Linear(state_dim, 128),
    nn.ReLU(),
    nn.Linear(128, 128),
    nn.ReLU(),
    nn.Linear(128, action_dim)
)
trained_model.load_state_dict(torch.load("trained_model.pth", map_location="cpu"))
trained_model.eval()

def evaluate_model(model):
    """Evaluate the trained model on all possible A-B configurations and specified agent distributions."""
    GRID_SIZE = 5
    distributions = [(1,3), (2,2), (3,1), (4,0)]  # (agents at B, agents at A)
    success_count = 0
    total_tests = 0
    stats = {
        'total_tests': 0,
        'successful_tests': 0,
        'failures': {'collisions': 0, 'timeout': 0, 'incomplete': 0},
        'distribution_success': {'1,3': 0, '2,2': 0, '3,1': 0, '4,0': 0},
        'avg_steps_successful': 0
    }
    # Helper: choose greedy action using the trained model
    def greedy_action(state):
        state_t = torch.tensor(state, dtype=torch.float32)
        with torch.no_grad():
            q_vals = model(state_t)
        return int(torch.argmax(q_vals).item())

    # Iterate over all possible A, B placements
    for ax in range(GRID_SIZE):
        for ay in range(GRID_SIZE):
            for bx in range(GRID_SIZE):
                for by in range(GRID_SIZE):
                    if (ax, ay) == (bx, by):
                        continue  # skip invalid configs where A == B
                    for (b_agents, a_agents) in distributions:
                        total_tests += 1
                        stats['total_tests'] = total_tests
                        # Initialize env with given A and B
                        test_env = GridWorldEnvironment(n=GRID_SIZE, m=GRID_SIZE, num_agents=4,
                                                        food_source_location=(ax, ay),
                                                        nest_location=(bx, by))
                        # Manually set agents at B or A according to the distribution
                        # First agent (index 0) at B, since we test that agent's ability to do B->A->B
                        for i in range(test_env.num_agents):
                            if i < b_agents:
                                # place this agent at B (nest), without item
                                test_env.agents[i].position = test_env.nest_location
                                test_env.agents[i].has_item = 0
                                test_env.agents[i].direction = False
                            else:
                                # place this agent at A (food source), with item
                                test_env.agents[i].position = test_env.food_source_location
                                test_env.agents[i].has_item = 1
                                test_env.agents[i].direction = True
                        # Reset central clock and counters for safety
                        test_env.clock = 0
                        test_env.collision_count = 0
                        test_env.round_trip_count = 0
                        for i in range(test_env.num_agents):
                            test_env.agent_states[i] = 0
                            test_env.agent_round_trips[i] = False

                        # Simulate until first agent (index 0) completes B->A->B or until 25 steps per agent
                        agent0_success = False
                        collided = False
                        max_steps = 25  # max moves per agent
                        steps_taken = 0
                        for t in range(max_steps * test_env.num_agents):
                            aid = test_env.get_next_agent()
                            state = test_env.get_state(aid)
                            action = greedy_action(state)
                            test_env.take_action(aid, action)
                            test_env.check_collisions()
                            # If any collision occurs, note if agent0 was involved
                            if test_env.collision_count > 0:
                                for ag in test_env.agents:
                                    if ag.collision_penalty != 0:
                                        if ag.agent_id == 0:
                                            collided = True
                                        ag.collision_penalty = 0
                            # Check if agent 0 completed B->A->B (round trip)
                            # B->A->B is completed when agent0 goes from B (start) -> A (pickup) -> B (drop-off).
                            # In our env logic, that corresponds to agent_states[0] reaching 3 (completed) or round_trip_count increment.
                            if test_env.agent_round_trips[0]:
                                agent0_success = True
                                break
                            steps_taken += 1
                        # Determine success/failure for this test
                        if agent0_success and not collided:
                            success_count += 1
                            stats['successful_tests'] += 1
                            stats['distribution_success'][f'{b_agents},{a_agents}'] += 1
                            stats['avg_steps_successful'] += steps_taken
                        else:
                            # Determine failure reason
                            if not agent0_success and steps_taken >= max_steps * test_env.num_agents:
                                stats['failures']['timeout'] += 1
                            elif collided:
                                stats['failures']['collisions'] += 1
                            else:
                                stats['failures']['incomplete'] += 1

    # Calculate success rate and average steps
    success_rate = success_count / total_tests
    if stats['successful_tests'] > 0:
        stats['avg_steps_successful'] /= stats['successful_tests']
    return success_rate, stats

# Run evaluation
success_rate, eval_stats = evaluate_model(trained_model)
print(f"Success Rate: {success_rate*100:.2f}%")
print("Success rate per distribution:", {dist: eval_stats['distribution_success'][dist] for dist in eval_stats['distribution_success']})
print("Failures breakdown:", eval_stats['failures'])
print(f"Average steps for successful episodes: {eval_stats['avg_steps_successful']:.2f}")